In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from Bio import SeqIO
from collections import Counter
from threading import Thread,Lock
import os
from time import sleep
counts = Counter()

In [2]:
data_path = '/media/wvdon/sdata/covid_evolution/sequence/msa_2022-10-09/2022-10-09_unmasked.fa'
meta_data_path = '/media/wvdon/sdata/covid_evolution/sequence/metadata_tsv_2022_10_10/metadata.tsv'

In [3]:
#total sequence 13396972

In [4]:
import pymysql
conn = pymysql.connect(
    host='127.0.0.1',
    user='sars',
    password='sarspassword',
    database='sars',
    charset='utf8',
    # autocommit=True,    # 如果插入数据，， 是否自动提交? 和conn.commit()功能一致。
)

In [5]:
def insert_data_struct():
    global data

    cursor= conn.cursor()
    # 插入sql语句
    sql = "insert into NSME_final (accession_id,N,S,E,M) values (%s,%s,%s,%s,%s)"
    for li in data:
        accession_id = str(li[0])
        N = str(li[1])
        S = str(li[2])
        E = str(li[3])
        M  = str(li[4])
#         collection_date = str(li[5])
#         pango_lineage = str(li[6])
#         location = str(li[7])
        insert = cursor.execute(sql,(accession_id,N,S,E,M))
    cursor.close()
    conn.commit()
    #conn.close()
    data = []
    #print("insert ok")

In [ ]:
meta_data = pd.read_csv(meta_data_path, sep='\t', header=0)#.set_index(['Accession ID'])

In [ ]:
meta_data[meta_data['Accession ID']=='EPI_ISL_402124']

In [ ]:
index_meta = meta_data.set_index(['Accession ID'])

In [ ]:
#index_meta
meta = index_meta.loc['EPI_ISL_402124']

In [28]:
meta['Pango lineage']

'B'

In [27]:
meta

Virus name                                            hCoV-19/Wuhan/WIV04/2019
Last vaccinated                                                            NaN
Passage details/history                                               Original
Type                                                           betacoronavirus
Collection date                                                     2019-12-30
Location                                          Asia / China / Hubei / Wuhan
Additional location information    Retailer at Huanan Seafood Wholesale Market
Sequence length                                                          29891
Host                                                                     Human
Patient age                                                                 49
Gender                                                                  Female
Clade                                                                        L
Pango lineage                                       

In [19]:
#print(len(meta_data))
#meta_data.head()
#EPI_ISL_402124

13427526


,Virus name,Last vaccinated,Passage details/history,Type,Collection date,Location,Additional location information,Sequence length,Host,Patient age,...,Pangolin version,Variant,AA Substitutions,Submission date,Is reference?,Is complete?,Is high coverage?,Is low coverage?,N-Content,GC-Content
Accession ID,,,,,,,,,,,,,,,,,,,,,
EPI_ISL_3385701,hCoV-19/USA/AZ-ASU10316/2021,NaN,Original,betacoronavirus,2021-08-02,North America / USA / Arizona / Coconino,NaN,29767,Human,unknown,...,PLEARN-v1.14,VOC Delta GK (B.1.617.2+AY.*) first detected i...,"(N_G215C,NS8_L60F,N_D63G,N_R203M,NSP12_G671S,N...",2021-08-13,NaN,True,NaN,True,0.062458,0.380577
EPI_ISL_9979246,hCoV-19/France/IDF-HMN-22012170861/2022,NaN,Original,betacoronavirus,2022-01-17,Europe / France / Ile-de-France,NaN,29823,Human,26,...,consensus call,VOC Omicron GRA (B.1.1.529+BA.*) first detecte...,"(Spike_T95I,NSP5_V204A,Spike_A67V,Spike_K417N,...",2022-02-16,NaN,True,NaN,True,0.131422,0.377090
EPI_ISL_5122902,hCoV-19/Germany/BY-RKI-I-281278/2021,NaN,Original,betacoronavirus,2021-10-05,Europe / Germany / Bavaria,NaN,29860,Human,unknown,...,PLEARN-v1.14,VOC Delta GK (B.1.617.2+AY.*) first detected i...,"(N_G215C,N_R203M,NSP12_G671S,NS3_S26L,Spike_P2...",2021-10-14,NaN,True,True,NaN,0.000201,0.378877
EPI_ISL_9979247,hCoV-19/France/GES-HMN-22022010589/2022,NaN,Original,betacoronavirus,2022-01-24,Europe / France / Grand Est,NaN,29745,Human,41,...,PLEARN-v1.14,VOC Omicron GRA (B.1.1.529+BA.*) first detecte...,"(NSP5_P132H,Spike_H69del,Spike_T95I,Spike_A67V...",2022-02-16,NaN,True,NaN,True,0.122911,0.378665
EPI_ISL_9979248,hCoV-19/France/CVL-HMN-22012270874/2022,NaN,Original,betacoronavirus,2022-01-03,Europe / France / Centre-Val de Loire,NaN,29769,Human,64,...,PLEARN-v1.14,VOC Delta GK (B.1.617.2+AY.*) first detected i...,"(N_G215C,Spike_T95I,N_D63G,N_R203M,NSP12_G671S...",2022-02-16,NaN,True,NaN,NaN,0.023380,0.379734


In [ ]:
import sarscov2
# import imp
# imp.reload(sarscov2)

In [ ]:
i = 0
data = []
count_error = 0
for seq_record in SeqIO.parse(data_path, "fasta"):
    
    meta = index_meta.loc[seq_record.id]

    i = i + 1
    E,M,N,S = sarscov2.NSEM_protein(seq_record.seq)
    #accession_id,N,S,E,M,collection_date,pango_lineage,location
    data.append([seq_record.id,N,S,E,M])
    if i %10000==0:
        print('%.2f' % i*100/13396972)
        insert_data_struct()
        data = []
insert_data_struct()
print('done')